In [2]:
import numpy as np
import pprint
import math

In [3]:
def begin_poker(players,hole_cards,community_cards):
    current_deck = deck()
    player_hands = poker_hand(players,hole_cards+community_cards,current_deck )
   
    current_deck.shuffle()
    unassigned_hands = current_deck.deal(players,hole_cards)
    for player_index in range(players):
        player_hands.cards[player_index,0:hole_cards] = unassigned_hands[player_index]
    
    
    return current_deck,player_hands

In [4]:
suit_list = ('Spades','Clubs','Diamonds','Hearts')
value_list = list(range(2,11)) +['Jack','Queen','King','Ace']
class deck:
    def __init__(self):
        self.cards = [None]*52
        self.order = np.array(range(52),dtype=int)
        self.suits = {};
        self.values = {};
        index=0;
        suit_index = 0;
        current_value =0;
        for suit in suit_list:
            self.suits[suit] = suit_index;
            suit_index +=1
            for value in value_list:
                self.cards[index] = [str(value), suit]
                if suit_index==1:
                    current_value += 1
                    self.values[str(value)] = current_value
                index +=1
    def shuffle(self,*args):
        if not args:
            print('no prior hands')
        else:
            hands = args[0]
            self.order = np.concatenate((self.order, hands.flatten()), axis=0) 
        np.random.shuffle(self.order)
    def deal(self,num_players,cards_per_hand):
        hands =np.zeros([num_players,cards_per_hand],dtype=int)
        for index in range(num_players):
            for card_index  in range(cards_per_hand):
                hands[index,card_index] = self.order[0]
                self.order = self.order[1:]
        return hands
    def deal_community(self,player_hands,cards):
        current_card_count =  np.bitwise_not(np.isnan(player_hands.cards[0]))
        current_card_count = np.nansum(current_card_count)
        if current_card_count+cards> player_hands.cards.shape[1]:
            print('can\'t add that many cards' )
            return
        
        for i in np.arange(current_card_count,current_card_count+cards):
            for player_num in range(len(player_hands.cards[:,0])):
                player_hands.cards[player_num,i] = self.order[0]
            self.order = self.order[1:]
            
        

In [32]:
class poker_hand:
    def __init__(self,players,total_cards,a_deck):
        self.cards = np.empty((players,total_cards),dtype=int)*np.nan
        self.pair_total = np.zeros(len(list(a_deck.values.values())))
        self.straight_check = np.empty
        self.straight_anchor = np.empty
        self.BestHand = np.empty((players,total_cards),dtype=int)*np.nan
        self.unique_values = np.empty
    def display(self,player,deck):
        current_cards = np.isnan(self.cards[player])
        current_cards =np.bitwise_not(current_cards)
        current_cards = self.cards[player][current_cards]
        current_cards  =current_cards.astype(int)
        current_cards = [ deck.cards[i] for i in current_cards]
        
        print(current_cards)
        #return(current_cards)
    def determine_best_hand(self,player_number,deck):
        player_hands.check_for_flush(player_number,deck)
        
        
        pair_info = player_hands.check_for_pairs(player_number,deck)
        
        
        straight_info = player_hands.check_for_straight(player_number,deck)
        flush_info = player_hands.check_for_flush(player_number,deck)
        return pair_info,straight_info,flush_info
    
    def check_for_pairs(self,player,deck):
        current_cards = np.isnan(self.cards[player]) #get rid of card slots that have not be dealt
        current_cards =np.bitwise_not(current_cards)
        current_cards = self.cards[player][current_cards] #get list of cards that have been dealt
        current_cards  =current_cards.astype(int) #change data type so that it can be used as an list of indices
      
        current_cards = [ deck.cards[i][0] for i in current_cards] #for the current cards IDs, get the card value (e.g, 2 or King)
        
        all_values = np.array(list(a_deck.values.values()))  #get ordered list of numeric values from dictionary
      
        #use the card values to get the numeric value from the dictionary
        #(e.g., Jack =10)
        current_values = [deck.values[str(i)] for i in current_cards]  
        
    
        ids, indices = np.unique(current_values, return_index = True)
        
        #unique list of current keys (e.g., King, Ace, 7)
        unique_keys = [current_cards[i] for i in indices]
        #list of values for unique keys
        unique_values = np.unique(current_values)
        self.unique_values = unique_values
        #grid to dtermine pairs
        x, y = np.meshgrid(unique_values, current_values)
        
        #determine which values match
        pair_logical = x==y
        pair_total = sum(pair_logical)
        self.pair_total = pair_total #save the value for future reference
     
        pair_count = [0,0,0];
        
        if max(self.pair_total)>1: # 1 indicates the value only matches itself

            for pair_index in range(len(self.pair_total)):
                
                if self.pair_total[pair_index]>1:
                    pair_value = current_cards[pair_index]
                   
                    #print('player ' + str(player) + ' has ' + str(self.pair_total[pair_index]) + ' ' + str(unique_keys[pair_index]) +'s')
                    if self.pair_total[pair_index]==2:
                        pair_count[0] += 1
                    elif  self.pair_total[pair_index]==3:
                        pair_count[1] += 1
                    elif self.pair_total[pair_index]==3:
                        pair_count[2] += 1
        
        
        non_paired = [unique_values[i] for i in range(len(unique_values)) if self.pair_total[i] == 1] 
        pair_info = [self.pair_total,unique_keys, max(non_paired)]
        [unique_values[i] for i in range(len(unique_values))]
        #if pair_count[0]>0 and pair_count[1]>0:
            #print('player ' + str(player) + ' has a full house')
        return pair_info
            
    def check_for_flush(self,player,deck):
        current_cards = np.isnan(self.cards[player])
        current_cards =np.bitwise_not(current_cards)
        current_cards = self.cards[player][current_cards]
        current_cards  =current_cards.astype(int)
        card_values = [ deck.cards[i][0] for i in current_cards]
       
        #all_values = np.array(list(a_deck.values.values()))
        #current_values = [deck.values[str(i)] for i in current_cards]
        
        
        current_cards = [ deck.cards[i][1] for i in current_cards]
     
        all_suits = np.array(list(a_deck.suits.values()))
        
        current_suits = [deck.suits[str(i)] for i in current_cards]
        
        x, y = np.meshgrid(np.array(list(a_deck.suits.values())), current_suits)
     
        current_flush = x==y

        flush_suit= None
        flush_cards = None
   
        if max(sum(current_flush))<5:
            #print('no flush  = ' + str(max(sum(current_flush))))
            flush_info = [0,'no flush']
        else:
            #print('we have a flush')
            suit_logical = sum(current_flush)>2
            flush_suit = list(a_deck.suits.keys())
            flush_suit = [suit_list[i] for i, x in enumerate(suit_logical) if x]
            flush_cards = [s for s in range(len(current_cards)) if current_cards[s]==flush_suit[0]]
            flush_values = [card_values[i] for i in flush_cards]
            flush_info = [1,flush_suit,flush_values]
            
        return flush_info
    def check_for_straight(self,player,deck):
        current_cards = np.isnan(self.cards[player])
        current_cards =np.bitwise_not(current_cards)
        current_cards = self.cards[player][current_cards]
        current_cards  =current_cards.astype(int)
      
        current_cards = [ deck.cards[i][0] for i in current_cards]
        
        all_values = np.array(list(a_deck.values.values()))
        
        current_values = [deck.values[str(i)] for i in current_cards]
        unique_values = np.unique(current_values)
        unique_values2 = unique_values[unique_values<10]
        if len(unique_values2)==0:
            #print('no valid anchors: will exit')
            self.straight_check =[]
            straight_info = [0, 'no straight']
            
            return straight_info

        x, y = np.meshgrid(unique_values2, unique_values)
        straight_check = (x-y)
        
        straight_check=np.logical_and(straight_check>=-4, straight_check<=0)
        
        straight_check=sum(straight_check)
        
        straight_check = straight_check 
        
        self.straight_check = straight_check
        
        self.straight_anchor = unique_values2
        #print('straight anchor = ' + str(self.straight_anchor))
        #print('straight check = ' + str(straight_check))
        
        if max(straight_check)<=2:
            straight_info = [0, 'no straight']
        else:
            #print('we have a straight')
            straight_anchor = [self.straight_anchor[i] for i in range(len(straight_check)) if straight_check[i]>4]
            straight_info = [1, straight_check, straight_anchor]
        return straight_info
    def straight_odds(self,player_number,deck):
        
        self.check_for_straight(player_number,deck)
        print(self.straight_check)
        if len(self.straight_check)==0:
            print('No valid anchors, will exit')
            return
        
        straight_check = self.straight_check

        deck_cards_remain = len(deck.order)
        hand_cards_remain = self.cards.shape[1] - len(straight_check)
        match_outcomes = np.zeros(len(straight_check))
        nonmatch_outcomes = np.zeros(len(straight_check))
        all_outcomes = math.factorial(deck_cards_remain)/(math.factorial(deck_cards_remain - hand_cards_remain)* \
                       math.factorial(hand_cards_remain))
        correction_outcomes = np.zeros(len(straight_check))

        if max(straight_check)>4:
            straight_odds = 1.0;
        else:
            for index  in range(len(straight_check)):
                current_match_cards = (5 - straight_check[index])
                current_non_match_cards = deck_cards_remain - (current_match_cards*4)
                non_match_slots = hand_cards_remain - (5 - straight_check[index])
                
                    
                if non_match_slots>=0:
                    match_outcomes[index]  = (4**current_match_cards)
            
                    nonmatch_outcomes[index] = math.factorial(current_non_match_cards)/(math.factorial(current_non_match_cards - \
                    non_match_slots)*math.factorial(non_match_slots))
                    if index >0:
                        independence_check = self.straight_anchor[index] - self.straight_anchor[0:index]
                        
                        if min(independence_check)<5:
                            print('dependent anchors: ' + str(independence_check))
                            #correction_outcomes[index] = 4**(current_match_cards-independence_check[index])
                        else:
                            print('independent anchors' + str(independence_check))
                    
                
                else:
                    match_outcomes[index]=0
                    match_outcomes *= nonmatch_outcomes       
                    straight_odds= match_outcomes/all_outcomes
        print(straight_odds)
        
        #add code to deal with non-independent straights

        

In [33]:
player_number = 5
community_cards = 5
a_deck,player_hands = begin_poker(player_number,2,community_cards)
for index in range(community_cards):
    a_deck.deal_community(player_hands,1)

print('cards dealt')
for index in range(player_number):
    player_hands.display(index,a_deck)


no prior hands
cards dealt
[['King', 'Diamonds'], ['2', 'Hearts'], ['6', 'Diamonds'], ['Ace', 'Diamonds'], ['10', 'Diamonds'], ['8', 'Clubs'], ['10', 'Spades']]
[['Ace', 'Clubs'], ['5', 'Hearts'], ['6', 'Diamonds'], ['Ace', 'Diamonds'], ['10', 'Diamonds'], ['8', 'Clubs'], ['10', 'Spades']]
[['7', 'Clubs'], ['2', 'Diamonds'], ['6', 'Diamonds'], ['Ace', 'Diamonds'], ['10', 'Diamonds'], ['8', 'Clubs'], ['10', 'Spades']]
[['King', 'Spades'], ['5', 'Spades'], ['6', 'Diamonds'], ['Ace', 'Diamonds'], ['10', 'Diamonds'], ['8', 'Clubs'], ['10', 'Spades']]
[['Jack', 'Hearts'], ['4', 'Clubs'], ['6', 'Diamonds'], ['Ace', 'Diamonds'], ['10', 'Diamonds'], ['8', 'Clubs'], ['10', 'Spades']]


In [19]:
pair_info = [None]*player_number
straight_info = [None]*player_number
flush_info = [None]*player_number
flush_count =  np.zeros(player_number,dtype=int)
straight_count =  np.zeros(player_number,dtype=int)
straight_anchor = np.zeros(player_number,dtype=int)
quad_max = np.zeros(player_number,dtype=int)
quad_count = np.zeros(player_number,dtype=int)
trip_max = np.zeros(player_number,dtype=int)
trip_count = np.zeros(player_number,dtype=int)
pair_max = np.zeros(player_number,dtype=int)
pair_count = np.zeros(player_number,dtype=int)
pair_kicker = np.zeros(player_number,dtype=int)
for index in range(player_number):
    pair_info[index],straight_info[index],flush_info[index] =player_hands.determine_best_hand(index,a_deck)
for index in range(player_number):
        flush_count[index] = flush_info[index][0] 
         
        straight_count[index] = straight_info[index][0] 
        print(straight_info[index])
        straight_anchor[index] = np.array(max(straight_info[index][2]))
        x =sum(pair_info[index][0]==2)
        pair_count[index]=x
        pair_kicker[index] = pair_info[index][2]
        x =sum(pair_info[index][0]==3)
        trip_count[index]=x
        
        x =sum(pair_info[index][0]==4)
        quad_count[index]=x
        
        current_pair = [pair_info[index][1][i] for i in range(len(pair_info[index][1])) if pair_info[index][0][i]==2]
        current_trip = [pair_info[index][1][i] for i in range(len(pair_info[index][1])) if pair_info[index][0][i]==3]
        if len(current_pair)>0:
            pair_max[index] = max([a_deck.values[i] for i in current_pair])
        if len(current_trip)>0:
            trip_max[index] = max([a_deck.values[i] for i in current_trip])

#straight flush is the highest hand
print('straight count is ' + str(straight_count))
print('flush count is ' + str(flush_count))
if max(straight_count)>0:
    result = np.where(straight_count == np.amax(straight_count))
    if len(result[0])==1:
        print('player ' + str(result[0]) + ' is the winner with a straight')
    else:
        result_straight_fush = np.logical_and(straight_count >0, flush_count >0)
        if len(result_straight_fush)==1:
             print('player ' + str(result_straight_fush) + ' is the winner with a straight flush')
        elif len(result_straight_fush)>1:
            result_tie_breaker = np.where(flush_count >0 and straight_count >0 & straight_count == np.amax(straight_count))
            print('straight tie breaker = ' + str(straight_anchor))
            winner = straight_anchor[result_tie_breaker]==max(straight_anchor)
            winner = np.array([result[0][i] for i in range(len(winner)) if winner[i]==True])
            if len(winner)==1:
                print('player ' + str(winner) + ' is the winner with a high card straight flush ')
            else:
                print('players ' + str(winner) + ' split the pot with straight flushes')
        else:
      
            print('straight tie breaker = ' + str(straight_anchor))
            winner = straight_anchor[result]==max(straight_anchor)
            winner = np.array([result[0][i] for i in range(len(winner)) if winner[i]==True])
            if len(winner)==1:
                print('player ' + str(winner) + ' is the winner with a high card straight')
            else:
                print('players ' + str(winner) + ' split the pot with straights')
            
#print('pair_count = ' + str(pair_count))
pair_count[pair_count>2] =2
#trip_count = np.array([0,1,2,2])
if max(quad_count)>0:
    result = np.where(quad_count == np.amax(quad_count))
    if len(result)==1:
        print('player ' + str(result[0]) + ' is the winner with quads')
elif max(trip_count)>0:
    result = np.where(trip_count == np.amax(trip_count))
    if len(result[0])==1:
        print('player ' + str(result[0]) + ' is the winner with trips')
    else:
        print(trip_count)
        print(result[0])
        print(trip_max[result[0]])
elif max(pair_count)>0:
    result = np.where(pair_count == np.amax(pair_count))
    print('result = ' + str(result))
    if len(result[0])==1:
        print('player ' + str(result[0]) + ' is the winner with pairs')
    else:
        print('pair max tie breaker = ' + str(pair_max))
        winner = pair_max[result]==max(pair_max)
        winner = np.array([result[0][i] for i in range(len(winner)) if winner[i]==True])
        print(winner)
        if len(winner)==1:
            print('player number ' +  str(winner) + ' is the winner with the highest pair')
        else:
            print('find high card kicker')
            result = np.where(winner == np.amax(winner))
            winner = winner[result[0]]
            print('player number ' +  str(winner) + ' is the winner with the highest pair and kicker')
#result = numpy.where(quad_count == numpy.amax(quad_count))
print('number of pairs = ' + str(pair_count))        
print('number of trips = ' + str(trip_count))  
print('number of quads = ' + str(quad_count))   
print('max pair id = ' + str(pair_max))

[1, array([5, 5, 4, 3, 2]), [5, 6]]
[1, array([5, 4, 3, 2, 2]), [5]]
[1, array([2, 5, 4, 3, 2, 1]), [5]]
[1, array([5, 4, 4, 3, 2]), [5]]
[1, array([3, 5, 5, 4, 3, 2, 1]), [4, 5]]
straight count is [1 1 1 1 1]
flush count is [1 1 1 1 1]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [31]:

np.logical_and((straight_count >0), (flush_count >0))

array([ True,  True,  True,  True,  True])

In [124]:
player_number = 0
player_hands.display(player_number,a_deck)
pair_info = player_hands.check_for_pairs(player_number,a_deck)
print(pair_info)

[['8', 'Spades'], ['Jack', 'Hearts'], ['Queen', 'Hearts'], ['Ace', 'Diamonds'], ['King', 'Hearts'], ['Queen', 'Diamonds'], ['2', 'Hearts']]
[array([1, 1, 1, 2, 1, 1]), ['2', '8', 'Jack', 'Queen', 'King', 'Ace'], 13]


In [125]:
pair_count

array([1])

In [646]:
player_number = 0
a_deck.deal_community(player_hands,1)
player_hands.display(player_number,a_deck)
#player_hands.check_for_pairs(player_number,a_deck)
flush_info = player_hands.check_for_flush(player_number,a_deck)
#player_hands.check_for_straight(player_number,a_deck)

[[6, 'Hearts'], ['Jack', 'Clubs'], ['King', 'Clubs'], [9, 'Hearts'], ['Queen', 'Diamonds']]


In [450]:
flush_info

[1, ['Clubs'], [7, 'Jack', 6]]

In [417]:
print(current_cards)
print(flush_suit)
#[s for s in range(len(current_cards)) if current_cards[s]==flush_suit]

current_cards[2]==flush_suit[0]

['Clubs', 'Diamonds', 'Spades', 'Spades', 'Diamonds', 'Spades']
['Spades']


True

In [287]:
player_number = 2
player_hands.display(player_number,a_deck)
player_hands.check_for_pairs(player_number,a_deck)
player_hands.check_for_flush(player_number,a_deck)
player_hands.check_for_straight(player_number,a_deck)

[[2, 'Spades'], [2, 'Hearts'], [6, 'Clubs']]
player 2 has 2 2s
player 2 has 1 pairs
player 2 has 0 trips
player 2 has 0 quads
no flush
no straight


In [132]:
player_hands.straight_odds(0,a_deck)


we have a straight
[3 4 5 4 3]
[0. 0. 0. 0. 0.]


In [62]:
player_hands.display(0,a_deck)

[[5, 'Hearts'], [10, 'Hearts']]


In [102]:
a = np.array([1,2])
a=[]
len(a)

0

In [158]:
player_hands.display(2,a_deck)
player_hands.check_for_flush(2,a_deck)

[['Jack', 'Clubs'], [3, 'Diamonds'], [5, 'Diamonds'], ['King', 'Clubs']]
[1, 2, 2, 1]
no flush


In [125]:
player_hands.check_for_pairs(2,a_deck)

[3, 9, 'Ace']
no current pairs


In [27]:
a_deck.suits

{'Spades': 0, 'Clubs': 1, 'Diamonds': 2, 'Hearts': 3}

In [33]:
np.array(list(a_deck.suits.values()))

array([0, 1, 2, 3])